# AP Research Data Processing Notebook 4: Determine Optimal Electrode For Electroreduction
The following codes in this Jupyter notebook (E0.ipynb) contains the data analysis portion of Benjamin Chang's AP Research project on *Optimal Conditions For Small-Scale Electroreduction of Common Non-Steroidal Anti-Inflammatory Drugs*. All charts, graReferenceNums, and figures are produced from the raw data in .csv files and manipulated by the following codes.

## Initializing Libraries and Style

In [2513]:
#importing libraries
import pandas as pd
import numpy as np
import scipy.stats
import math
from matplotlib import pyplot
from IPython.display import display, HTML

In [2514]:
font = {
    'family' : 'georgia',
    'color': 'black',
    'weight': 'normal',
    'size': 11
}

## Salicylic Acid

In [2515]:
#Merging the data into one dataframe
csv_file_list = ['E3-SA/E3-SA-Aluminum.csv', 'E3-SA/E3-SA-Copper.csv', 'E3-SA/E3-SA-Iron.csv', 'E3-SA/E3-SA-Platinum.csv', 'E3-SA/E3-SA-Zinc.csv',]

list_of_dataframes = []

for filename in csv_file_list:
    list_of_dataframes.append(pd.read_csv(filename))

merged_df = pd.concat(list_of_dataframes)

In [2516]:
#Find the percent difference between Absorbance Before and Absorbance After
merged_df['Percent Difference (%)'] = (merged_df['Absorbance After (Au)'] - merged_df['Absorbance Before (Au)'])/merged_df['Absorbance Before (Au)']
merged_df['Percent Difference (%)'] = merged_df['Percent Difference (%)']*100

#Find difference between Absorbance Before and Absorbance After
merged_df['Mean Difference'] = merged_df['Absorbance After (Au)'] - merged_df['Absorbance Before (Au)']

In [2517]:
#Create a numpy array of standard deviation and store it in the dataframe
std = []
for i in range(1, 6):
    std.append(merged_df.loc[merged_df['ReferenceNum'] == i]['Mean Difference'].std())

arr = np.array(std)

#Compute the average of every 5 rows and condense the dataframe
merged_df = merged_df.groupby(np.arange(len(merged_df))//5).mean()
df = pd.DataFrame({'Standard Deviation': arr})

#Join two dataframes
merged_df = pd.concat([merged_df, df], axis=1, join='inner')

In [2518]:
#Compute the t-value (confidence level = 5%, df = 4)
t_value = scipy.stats.t.ppf(q=1-.05/2,df=4)

#Create a numpy array of intervals and store it into the dataframe
intervals = []
for i in range(1, 6):
    interval = t_value * merged_df.loc[merged_df['ReferenceNum'] == i]['Standard Deviation'] / math.sqrt(5)
    intervals.append(interval)
intervals = np.array(intervals, dtype='float')
#Squeeze into one dimension
intervals = np.squeeze(intervals)

#Create new dataframe
df2 = pd.DataFrame({'Interval': intervals}, index=[0, 1, 2, 3, 4])

#Join two dataframes
merged_df = pd.concat([merged_df, df2], axis=1, join='inner')

In [2519]:
#Create a numpy array of lower bounds and store it into the datagrame
lbd = []
for i in range(1, 6):
    interval = t_value * merged_df.loc[merged_df['ReferenceNum'] == i]['Standard Deviation'] / math.sqrt(5)
    mean = merged_df.loc[merged_df['ReferenceNum'] == i]['Mean Difference']
    lbd.append(mean - interval)
lbd = np.array(lbd, dtype='float')
#Squeeze into one dimension
lbd = np.squeeze(lbd)

#Create new dataframe
df = pd.DataFrame({'Lower Bound': lbd}, index=[0, 1, 2, 3, 4])

#Join two dataframes
merged_df = pd.concat([merged_df, df], axis=1, join='inner')

In [2520]:
#Create a numpy array of upper bounds and store it into the datagrame
ubd = []
for i in range(1, 6):
    interval = t_value * merged_df.loc[merged_df['ReferenceNum'] == i]['Standard Deviation'] / math.sqrt(5)
    mean = merged_df.loc[merged_df['ReferenceNum'] == i]['Mean Difference']
    ubd.append(mean + interval)
ubd = np.array(ubd, dtype='float')
#Squeeze into one dimension
ubd = np.squeeze(ubd)

#Create new dataframe
df1 = pd.DataFrame({'Upper Bound': ubd}, index=[0, 1, 2, 3, 4])

#Join two dataframes
merged_df = pd.concat([merged_df, df1], axis=1, join='inner')

In [2521]:
#Create a numpy array of upper bounds and store it into the datagrame
full = []
for i in range(1, 6):
    mean = merged_df.loc[merged_df['ReferenceNum'] == i]['Mean Difference'].round(5).apply(str)
    interval = merged_df.loc[merged_df['ReferenceNum'] == i]['Interval'].round(5).apply(str)
    # Store string 
    full.append(mean + ' ± ' + interval)
full = np.array(full, dtype='object')

# Squeeze into one dimension
full = np.squeeze(full)

#Create new dataframe
df = pd.DataFrame({'Confidence Interval': full}, index=[0, 1, 2, 3, 4])

#Join two dataframes
merged_df = pd.concat([merged_df, df], axis=1, join='inner')

#Remove trial column from dataframe
merged_df = merged_df.drop(['Trial (2min)'], axis=1)

#Save to new csv file
merged_df.to_csv('Processed Data/E3-SA-Processed.csv')

# Keep only the columns we need
save_dfSA = merged_df[['ReferenceNum', 'Percent Difference (%)', 'Mean Difference', 'Interval']]

#Output table as HTML
merged_df.to_html('Tables/E3-SA-Table.html')
merged_df

,ReferenceNum,Wavelength (nm),Absorbance Before (Au),Absorbance After (Au),Percent Difference (%),Mean Difference,Standard Deviation,Interval,Lower Bound,Upper Bound,Confidence Interval
0,5.0,330.0,0.16842,0.09860,-41.455886,-0.06982,0.000268,0.000222,-0.061042,-0.060598,-0.06082 ± 0.00033
1,4.0,330.0,0.16830,0.15350,-8.793763,-0.01480,0.000397,0.000263,-0.015203,-0.014677,-0.01494 ± 0.00049
2,3.0,330.0,0.16844,0.14636,-13.108024,-0.02208,0.000572,0.000710,-0.022790,-0.021370,-0.02208 ± 0.00071
3,2.0,330.0,0.16844,0.15350,-8.869338,-0.01494,0.000212,0.000494,-0.015294,-0.014306,-0.0148 ± 0.00026
4,1.0,330.0,0.16822,0.10740,-36.154962,-0.06082,0.000179,0.000333,-0.070153,-0.069487,-0.06982 ± 0.00022


## Ethyl Alcohol

In [2523]:
#Merging the data into one dataframe
csv_file_list = ['E3-EA/E3-EA-Aluminum.csv', 'E3-EA/E3-EA-Copper.csv', 'E3-EA/E3-EA-Iron.csv', 'E3-EA/E3-EA-Platinum.csv', 'E3-EA/E3-EA-Zinc.csv',]

list_of_dataframes = []

for filename in csv_file_list:
    list_of_dataframes.append(pd.read_csv(filename))

merged_df = pd.concat(list_of_dataframes)

In [2524]:
#Find the percent difference between Absorbance Before and Absorbance After
merged_df['Percent Difference (%)'] = (merged_df['Absorbance After (Au)'] - merged_df['Absorbance Before (Au)'])/merged_df['Absorbance Before (Au)']
merged_df['Percent Difference (%)'] = merged_df['Percent Difference (%)']*100

#Find difference between Absorbance Before and Absorbance After
merged_df['Mean Difference'] = merged_df['Absorbance After (Au)'] - merged_df['Absorbance Before (Au)']

In [2525]:
#Create a numpy array of standard deviation and store it in the dataframe
std = []
for i in range(1, 6):
    std.append(merged_df.loc[merged_df['ReferenceNum'] == i]['Mean Difference'].std())

arr = np.array(std)

#Compute the average of every 5 rows and condense the dataframe
merged_df = merged_df.groupby(np.arange(len(merged_df))//5).mean()
df = pd.DataFrame({'Standard Deviation': arr})

#Join two dataframes
merged_df = pd.concat([merged_df, df], axis=1, join='inner')

In [2526]:
#Compute the t-value (confidence level = 5%, df = 4)
t_value = scipy.stats.t.ppf(q=1-.05/2,df=4)

#Create a numpy array of intervals and store it into the dataframe
intervals = []
for i in range(1, 6):
    interval = t_value * merged_df.loc[merged_df['ReferenceNum'] == i]['Standard Deviation'] / math.sqrt(5)
    intervals.append(interval)
intervals = np.array(intervals, dtype='float')
#Squeeze into one dimension
intervals = np.squeeze(intervals)

#Create new dataframe
df2 = pd.DataFrame({'Interval': intervals}, index=[0, 1, 2, 3, 4])

#Join two dataframes
merged_df = pd.concat([merged_df, df2], axis=1, join='inner')

In [2527]:
#Create a numpy array of lower bounds and store it into the datagrame
lbd = []
for i in range(1, 6):
    interval = t_value * merged_df.loc[merged_df['ReferenceNum'] == i]['Standard Deviation'] / math.sqrt(5)
    mean = merged_df.loc[merged_df['ReferenceNum'] == i]['Mean Difference']
    lbd.append(mean - interval)
lbd = np.array(lbd, dtype='float')
#Squeeze into one dimension
lbd = np.squeeze(lbd)

#Create new dataframe
df = pd.DataFrame({'Lower Bound': lbd}, index=[0, 1, 2, 3, 4])

#Join two dataframes
merged_df = pd.concat([merged_df, df], axis=1, join='inner')

In [2528]:
#Create a numpy array of upper bounds and store it into the datagrame
ubd = []
for i in range(1, 6):
    interval = t_value * merged_df.loc[merged_df['ReferenceNum'] == i]['Standard Deviation'] / math.sqrt(5)
    mean = merged_df.loc[merged_df['ReferenceNum'] == i]['Mean Difference']
    ubd.append(mean + interval)
ubd = np.array(ubd, dtype='float')
#Squeeze into one dimension
ubd = np.squeeze(ubd)

#Create new dataframe
df1 = pd.DataFrame({'Upper Bound': ubd}, index=[0, 1, 2, 3, 4])

#Join two dataframes
merged_df = pd.concat([merged_df, df1], axis=1, join='inner')

In [2529]:
#Create a numpy array of upper bounds and store it into the datagrame
full = []
for i in range(1, 6):
    mean = merged_df.loc[merged_df['ReferenceNum'] == i]['Mean Difference'].round(5).apply(str)
    interval = merged_df.loc[merged_df['ReferenceNum'] == i]['Interval'].round(5).apply(str)
    # Store string 
    full.append(mean + ' ± ' + interval)
full = np.array(full, dtype='object')

# Squeeze into one dimension
full = np.squeeze(full)

#Create new dataframe
df = pd.DataFrame({'Confidence Interval': full}, index=[0, 1, 2, 3, 4])

#Join two dataframes
merged_df = pd.concat([merged_df, df], axis=1, join='inner')

#Remove trial column from dataframe
merged_df = merged_df.drop(['Trial (2min)'], axis=1)

#Save to new csv file
merged_df.to_csv('Processed Data/E3-EA-Processed.csv')

# Keep only the columns we need
save_dfEA = merged_df[['ReferenceNum', 'Percent Difference (%)', 'Mean Difference', 'Interval']]

#Output table as HTML
merged_df.to_html('Tables/E3-EA-Table.html')
merged_df

,ReferenceNum,Wavelength (nm),Absorbance Before (Au),Absorbance After (Au),Percent Difference (%),Mean Difference,Standard Deviation,Interval,Lower Bound,Upper Bound,Confidence Interval
0,5.0,330.0,0.45942,0.27148,-40.908045,-0.18794,0.000110,0.000783,-0.230863,-0.229297,-0.23008 ± 0.00014
1,4.0,330.0,0.45940,0.39534,-13.944256,-0.06406,0.000370,0.000417,-0.132037,-0.131203,-0.13162 ± 0.00046
2,3.0,330.0,0.45938,0.40232,-12.421056,-0.05706,0.000344,0.000427,-0.057487,-0.056633,-0.05706 ± 0.00043
3,2.0,330.0,0.45922,0.32760,-28.661653,-0.13162,0.000336,0.000460,-0.064520,-0.063600,-0.06406 ± 0.00042
4,1.0,330.0,0.45942,0.22934,-50.080544,-0.23008,0.000631,0.000136,-0.188076,-0.187804,-0.18794 ± 0.00078


## Citric Acid Monohydrate

In [2531]:
#Merging the data into one dataframe
csv_file_list = ['E3-CAM/E3-CAM-Aluminum.csv', 'E3-CAM/E3-CAM-Copper.csv', 'E3-CAM/E3-CAM-Iron.csv', 'E3-CAM/E3-CAM-Platinum.csv', 'E3-CAM/E3-CAM-Zinc.csv',]

list_of_dataframes = []

for filename in csv_file_list:
    list_of_dataframes.append(pd.read_csv(filename))

merged_df = pd.concat(list_of_dataframes)

In [2532]:
#Find the percent difference between Absorbance Before and Absorbance After
merged_df['Percent Difference (%)'] = (merged_df['Absorbance After (Au)'] - merged_df['Absorbance Before (Au)'])/merged_df['Absorbance Before (Au)']
merged_df['Percent Difference (%)'] = merged_df['Percent Difference (%)']*100

#Find difference between Absorbance Before and Absorbance After
merged_df['Mean Difference'] = merged_df['Absorbance After (Au)'] - merged_df['Absorbance Before (Au)']

In [2533]:
#Create a numpy array of standard deviation and store it in the dataframe
std = []
for i in range(1, 6):
    std.append(merged_df.loc[merged_df['ReferenceNum'] == i]['Mean Difference'].std())

arr = np.array(std)

#Compute the average of every 5 rows and condense the dataframe
merged_df = merged_df.groupby(np.arange(len(merged_df))//5).mean()
df = pd.DataFrame({'Standard Deviation': arr})

#Join two dataframes
merged_df = pd.concat([merged_df, df], axis=1, join='inner')

In [2534]:
#Compute the t-value (confidence level = 5%, df = 4)
t_value = scipy.stats.t.ppf(q=1-.05/2,df=4)

#Create a numpy array of intervals and store it into the dataframe
intervals = []
for i in range(1, 6):
    interval = t_value * merged_df.loc[merged_df['ReferenceNum'] == i]['Standard Deviation'] / math.sqrt(5)
    intervals.append(interval)
intervals = np.array(intervals, dtype='float')
#Squeeze into one dimension
intervals = np.squeeze(intervals)

#Create new dataframe
df2 = pd.DataFrame({'Interval': intervals}, index=[0, 1, 2, 3, 4])

#Join two dataframes
merged_df = pd.concat([merged_df, df2], axis=1, join='inner')

In [2535]:
#Create a numpy array of lower bounds and store it into the datagrame
lbd = []
for i in range(1, 6):
    interval = t_value * merged_df.loc[merged_df['ReferenceNum'] == i]['Standard Deviation'] / math.sqrt(5)
    mean = merged_df.loc[merged_df['ReferenceNum'] == i]['Mean Difference']
    lbd.append(mean - interval)
lbd = np.array(lbd, dtype='float')
#Squeeze into one dimension
lbd = np.squeeze(lbd)

#Create new dataframe
df = pd.DataFrame({'Lower Bound': lbd}, index=[0, 1, 2, 3, 4])

#Join two dataframes
merged_df = pd.concat([merged_df, df], axis=1, join='inner')

In [2536]:
#Create a numpy array of upper bounds and store it into the datagrame
ubd = []
for i in range(1, 6):
    interval = t_value * merged_df.loc[merged_df['ReferenceNum'] == i]['Standard Deviation'] / math.sqrt(5)
    mean = merged_df.loc[merged_df['ReferenceNum'] == i]['Mean Difference']
    ubd.append(mean + interval)
ubd = np.array(ubd, dtype='float')
#Squeeze into one dimension
ubd = np.squeeze(ubd)

#Create new dataframe
df1 = pd.DataFrame({'Upper Bound': ubd}, index=[0, 1, 2, 3, 4])

#Join two dataframes
merged_df = pd.concat([merged_df, df1], axis=1, join='inner')

In [2537]:
#Create a numpy array of upper bounds and store it into the datagrame
full = []
for i in range(1, 6):
    mean = merged_df.loc[merged_df['ReferenceNum'] == i]['Mean Difference'].round(5).apply(str)
    interval = merged_df.loc[merged_df['ReferenceNum'] == i]['Interval'].round(5).apply(str)
    # Store string 
    full.append(mean + ' ± ' + interval)
full = np.array(full, dtype='object')

# Squeeze into one dimension
full = np.squeeze(full)

#Create new dataframe
df = pd.DataFrame({'Confidence Interval': full}, index=[0, 1, 2, 3, 4])

#Join two dataframes
merged_df = pd.concat([merged_df, df], axis=1, join='inner')

#Remove trial column from dataframe
merged_df = merged_df.drop(['Trial (2min)'], axis=1)

#Save to new csv file
merged_df.to_csv('Processed Data/E3-CAM-Processed.csv')

# Keep only the columns we need
save_dfCAM = merged_df[['ReferenceNum', 'Percent Difference (%)', 'Mean Difference', 'Interval']]

#Output table as HTML
merged_df.to_html('Tables/E3-CAM-Table.html')
merged_df

,ReferenceNum,Wavelength (nm),Absorbance Before (Au),Absorbance After (Au),Percent Difference (%),Mean Difference,Standard Deviation,Interval,Lower Bound,Upper Bound,Confidence Interval
0,5.0,330.0,0.05736,0.03642,-36.503851,-0.02094,0.000371,0.000637,-0.014677,-0.013403,-0.01404 ± 0.00046
1,4.0,330.0,0.05752,0.05542,-3.650438,-0.00210,0.000152,0.000329,-0.015369,-0.014711,-0.01504 ± 0.00019
2,3.0,330.0,0.05748,0.04642,-19.241172,-0.01106,0.000230,0.000286,-0.011346,-0.010774,-0.01106 ± 0.00029
3,2.0,330.0,0.05752,0.04248,-26.147878,-0.01504,0.000265,0.000188,-0.002288,-0.001912,-0.0021 ± 0.00033
4,1.0,330.0,0.05758,0.04354,-24.382932,-0.01404,0.000513,0.000461,-0.021401,-0.020479,-0.02094 ± 0.00064


## Acetylsalicylic Acid

In [2539]:
#Merging the data into one dataframe
csv_file_list = ['E3-AA/E3-AA-Aluminum.csv', 'E3-AA/E3-AA-Copper.csv', 'E3-AA/E3-AA-Iron.csv', 'E3-AA/E3-AA-Platinum.csv', 'E3-AA/E3-AA-Zinc.csv',]

list_of_dataframes = []

for filename in csv_file_list:
    list_of_dataframes.append(pd.read_csv(filename))

merged_df = pd.concat(list_of_dataframes)

In [2540]:
#Find the percent difference between Absorbance Before and Absorbance After
merged_df['Percent Difference (%)'] = (merged_df['Absorbance After (Au)'] - merged_df['Absorbance Before (Au)'])/merged_df['Absorbance Before (Au)']
merged_df['Percent Difference (%)'] = merged_df['Percent Difference (%)']*100

#Find difference between Absorbance Before and Absorbance After
merged_df['Mean Difference'] = merged_df['Absorbance After (Au)'] - merged_df['Absorbance Before (Au)']

In [2541]:
#Create a numpy array of standard deviation and store it in the dataframe
std = []
for i in range(1, 6):
    std.append(merged_df.loc[merged_df['ReferenceNum'] == i]['Mean Difference'].std())

arr = np.array(std)

#Compute the average of every 5 rows and condense the dataframe
merged_df = merged_df.groupby(np.arange(len(merged_df))//5).mean()
df = pd.DataFrame({'Standard Deviation': arr})

#Join two dataframes
merged_df = pd.concat([merged_df, df], axis=1, join='inner')

In [2542]:
#Compute the t-value (confidence level = 5%, df = 4)
t_value = scipy.stats.t.ppf(q=1-.05/2,df=4)

#Create a numpy array of intervals and store it into the dataframe
intervals = []
for i in range(1, 6):
    interval = t_value * merged_df.loc[merged_df['ReferenceNum'] == i]['Standard Deviation'] / math.sqrt(5)
    intervals.append(interval)
intervals = np.array(intervals, dtype='float')
#Squeeze into one dimension
intervals = np.squeeze(intervals)

#Create new dataframe
df2 = pd.DataFrame({'Interval': intervals}, index=[0, 1, 2, 3, 4])

#Join two dataframes
merged_df = pd.concat([merged_df, df2], axis=1, join='inner')

In [2543]:
#Create a numpy array of lower bounds and store it into the datagrame
lbd = []
for i in range(1, 6):
    interval = t_value * merged_df.loc[merged_df['ReferenceNum'] == i]['Standard Deviation'] / math.sqrt(5)
    mean = merged_df.loc[merged_df['ReferenceNum'] == i]['Mean Difference']
    lbd.append(mean - interval)
lbd = np.array(lbd, dtype='float')
#Squeeze into one dimension
lbd = np.squeeze(lbd)

#Create new dataframe
df = pd.DataFrame({'Lower Bound': lbd}, index=[0, 1, 2, 3, 4])

#Join two dataframes
merged_df = pd.concat([merged_df, df], axis=1, join='inner')

In [2544]:
#Create a numpy array of upper bounds and store it into the datagrame
ubd = []
for i in range(1, 6):
    interval = t_value * merged_df.loc[merged_df['ReferenceNum'] == i]['Standard Deviation'] / math.sqrt(5)
    mean = merged_df.loc[merged_df['ReferenceNum'] == i]['Mean Difference']
    ubd.append(mean + interval)
ubd = np.array(ubd, dtype='float')
#Squeeze into one dimension
ubd = np.squeeze(ubd)

#Create new dataframe
df1 = pd.DataFrame({'Upper Bound': ubd}, index=[0, 1, 2, 3, 4])

#Join two dataframes
merged_df = pd.concat([merged_df, df1], axis=1, join='inner')

In [2545]:
#Create a numpy array of upper bounds and store it into the datagrame
full = []
for i in range(1, 6):
    mean = merged_df.loc[merged_df['ReferenceNum'] == i]['Mean Difference'].round(5).apply(str)
    interval = merged_df.loc[merged_df['ReferenceNum'] == i]['Interval'].round(5).apply(str)
    # Store string 
    full.append(mean + ' ± ' + interval)
full = np.array(full, dtype='object')

# Squeeze into one dimension
full = np.squeeze(full)

#Create new dataframe
df = pd.DataFrame({'Confidence Interval': full}, index=[0, 1, 2, 3, 4])

#Join two dataframes
merged_df = pd.concat([merged_df, df], axis=1, join='inner')

#Remove trial column from dataframe
merged_df = merged_df.drop(['Trial (2min)'], axis=1)

#Save to new csv file
merged_df.to_csv('Processed Data/E3-AA-Processed.csv')

# Keep only the columns we need
save_dfAA = merged_df[['ReferenceNum', 'Percent Difference (%)', 'Mean Difference', 'Interval']]

#Output table as HTML
merged_df.to_html('Tables/E3-AA-Table.html')
merged_df

,ReferenceNum,Wavelength (nm),Absorbance Before (Au),Absorbance After (Au),Percent Difference (%),Mean Difference,Standard Deviation,Interval,Lower Bound,Upper Bound,Confidence Interval
0,5.0,330.0,0.05836,0.05346,-8.394939,-0.00490,0.000387,0.000465,-0.020565,-0.019635,-0.0201 ± 0.00048
1,4.0,330.0,0.05842,0.04644,-20.504327,-0.01198,0.000430,0.000592,-0.050092,-0.048908,-0.0495 ± 0.00053
2,3.0,330.0,0.05830,0.05054,-13.310498,-0.00776,0.000261,0.000324,-0.008084,-0.007436,-0.00776 ± 0.00032
3,2.0,330.0,0.05886,0.00936,-84.096797,-0.04950,0.000476,0.000534,-0.012514,-0.011446,-0.01198 ± 0.00059
4,1.0,330.0,0.05846,0.03836,-34.381714,-0.02010,0.000374,0.000481,-0.005381,-0.004419,-0.0049 ± 0.00046


## AcetaminoReferenceNumen

In [2547]:
#Merging the data into one dataframe
csv_file_list = ['E3-A/E3-A-Aluminum.csv', 'E3-A/E3-A-Copper.csv', 'E3-A/E3-A-Iron.csv', 'E3-A/E3-A-Platinum.csv', 'E3-A/E3-A-Zinc.csv',]

list_of_dataframes = []

for filename in csv_file_list:
    list_of_dataframes.append(pd.read_csv(filename))

merged_df = pd.concat(list_of_dataframes)

In [2548]:
#Find the percent difference between Absorbance Before and Absorbance After
merged_df['Percent Difference (%)'] = (merged_df['Absorbance After (Au)'] - merged_df['Absorbance Before (Au)'])/merged_df['Absorbance Before (Au)']
merged_df['Percent Difference (%)'] = merged_df['Percent Difference (%)']*100

#Find difference between Absorbance Before and Absorbance After
merged_df['Mean Difference'] = merged_df['Absorbance After (Au)'] - merged_df['Absorbance Before (Au)']

In [2549]:
#Create a numpy array of standard deviation and store it in the dataframe
std = []
for i in range(1, 6):
    std.append(merged_df.loc[merged_df['ReferenceNum'] == i]['Mean Difference'].std())

arr = np.array(std)

#Compute the average of every 5 rows and condense the dataframe
merged_df = merged_df.groupby(np.arange(len(merged_df))//5).mean()
df = pd.DataFrame({'Standard Deviation': arr})

#Join two dataframes
merged_df = pd.concat([merged_df, df], axis=1, join='inner')

In [2550]:
#Compute the t-value (confidence level = 5%, df = 4)
t_value = scipy.stats.t.ppf(q=1-.05/2,df=4)

#Create a numpy array of intervals and store it into the dataframe
intervals = []
for i in range(1, 6):
    interval = t_value * merged_df.loc[merged_df['ReferenceNum'] == i]['Standard Deviation'] / math.sqrt(5)
    intervals.append(interval)
intervals = np.array(intervals, dtype='float')
#Squeeze into one dimension
intervals = np.squeeze(intervals)

#Create new dataframe
df2 = pd.DataFrame({'Interval': intervals}, index=[0, 1, 2, 3, 4])

#Join two dataframes
merged_df = pd.concat([merged_df, df2], axis=1, join='inner')

In [2551]:
#Create a numpy array of lower bounds and store it into the datagrame
lbd = []
for i in range(1, 6):
    interval = t_value * merged_df.loc[merged_df['ReferenceNum'] == i]['Standard Deviation'] / math.sqrt(5)
    mean = merged_df.loc[merged_df['ReferenceNum'] == i]['Mean Difference']
    lbd.append(mean - interval)
lbd = np.array(lbd, dtype='float')
#Squeeze into one dimension
lbd = np.squeeze(lbd)

#Create new dataframe
df = pd.DataFrame({'Lower Bound': lbd}, index=[0, 1, 2, 3, 4])

#Join two dataframes
merged_df = pd.concat([merged_df, df], axis=1, join='inner')

In [2552]:
#Create a numpy array of upper bounds and store it into the datagrame
ubd = []
for i in range(1, 6):
    interval = t_value * merged_df.loc[merged_df['ReferenceNum'] == i]['Standard Deviation'] / math.sqrt(5)
    mean = merged_df.loc[merged_df['ReferenceNum'] == i]['Mean Difference']
    ubd.append(mean + interval)
ubd = np.array(ubd, dtype='float')
#Squeeze into one dimension
ubd = np.squeeze(ubd)

#Create new dataframe
df1 = pd.DataFrame({'Upper Bound': ubd}, index=[0, 1, 2, 3, 4])

#Join two dataframes
merged_df = pd.concat([merged_df, df1], axis=1, join='inner')

In [2553]:
#Create a numpy array of upper bounds and store it into the datagrame
full = []
for i in range(1, 6):
    mean = merged_df.loc[merged_df['ReferenceNum'] == i]['Mean Difference'].round(5).apply(str)
    interval = merged_df.loc[merged_df['ReferenceNum'] == i]['Interval'].round(5).apply(str)
    # Store string 
    full.append(mean + ' ± ' + interval)
full = np.array(full, dtype='object')

# Squeeze into one dimension
full = np.squeeze(full)

#Create new dataframe
df = pd.DataFrame({'Confidence Interval': full}, index=[0, 1, 2, 3, 4])

#Join two dataframes
merged_df = pd.concat([merged_df, df], axis=1, join='inner')

#Remove trial column from dataframe
merged_df = merged_df.drop(['Trial (2min)'], axis=1)

#Save to new csv file
merged_df.to_csv('Processed Data/E3-A-Processed.csv')

# Keep only the columns we need
save_dfA = merged_df[['ReferenceNum', 'Percent Difference (%)', 'Mean Difference', 'Interval']]

#Output table as HTML
merged_df.to_html('Tables/E3-A-Table.html')
merged_df

,ReferenceNum,Wavelength (nm),Absorbance Before (Au),Absorbance After (Au),Percent Difference (%),Mean Difference,Standard Deviation,Interval,Lower Bound,Upper Bound,Confidence Interval
0,5.0,330.0,0.43230,0.33562,-22.364095,-0.09668,0.000141,0.000296,-0.083296,-0.082704,-0.083 ± 0.00018
1,4.0,330.0,0.43254,0.38444,-11.120359,-0.04810,0.000089,0.000263,-0.067323,-0.066797,-0.06706 ± 0.00011
2,3.0,330.0,0.43242,0.42862,-0.878722,-0.00380,0.000469,0.000582,-0.004382,-0.003218,-0.0038 ± 0.00058
3,2.0,330.0,0.43232,0.36526,-15.511654,-0.06706,0.000212,0.000111,-0.048211,-0.047989,-0.0481 ± 0.00026
4,1.0,330.0,0.43234,0.34934,-19.197862,-0.08300,0.000239,0.000176,-0.096856,-0.096504,-0.09668 ± 0.0003
